### Load and Use DeepForest for Object Detection

The DeepForest model is primarily used for detecting trees in aerial or satellite imagery. Here’s how you can use it:

In [ ]:
from deepforest import DeepForest
from deepforest import get_data
import cv2

# Load a pre-trained DeepForest model
model = DeepForest()
model.use_release()

# Load an image
image_path = get_data("../data/clean_data/solar/12011952/12011952_p_1_2022_06_02_rgb.tif")
image = cv2.imread(image_path)

# Perform object detection
predictions = model.predict_image(image=image)

#### Predict tiles instead of images
Large tiles covering wide geographic extents cannot fit into memory during prediction and would yield poor results due to the density of bounding boxes. Often provided as geospatial .tif files, remote sensing data is best suited for the predict_tile function, which splits the tile into overlapping windows, performs prediction on each of the windows, and then reassembles the resulting annotations.

Let’s show an example with a small image. For larger images, patch_size should be increased.

In [ ]:
raster_path = get_data("OSBS_029.tif")
# Window size of 300px with an overlap of 25% among windows for this small tile.
predicted_raster = model.predict_tile(raster_path, return_plot = True, patch_size=300,patch_overlap=0.25)

# View boxes overlayed when return_plot=True, when False, boxes are returned.
plt.imshow(predicted_raster)
plt.show()

### Use SAM for Segmentation
SAM can use boxes as input

In [ ]:
from segment_anything import SamPredictor, sam_model_registry
import torch

# Load the SAM model
sam_checkpoint = "sam_vit_h.pth"  # Update with your SAM model checkpoint
device = "cuda" if torch.cuda.is_available() else "cpu"
model_type = "vit_h"  # Example model type

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)
predictor = SamPredictor(sam)

# Set the image for segmentation
predictor.set_image(image)

### Combine Object Detection with Segmentation

Now, you can combine the outputs from DeepForest and SAM. You might want to segment each detected object (e.g., each tree) separately.

In [ ]:
import numpy as np

# Iterate through the detected objects and use SAM to segment them
for i, prediction in predictions.iterrows():
    # Extract the bounding box from the DeepForest predictions
    x1, y1, x2, y2 = int(prediction['xmin']), int(prediction['ymin']), int(prediction['xmax']), int(prediction['ymax'])

    # You can crop the image around the bounding box or directly use it as a prompt
    # Crop the image if necessary
    cropped_image = image[y1:y2, x1:x2]

    # Create a box prompt for SAM
    box = np.array([x1, y1, x2, y2])

    # Predict the mask using SAM
    masks, _, _ = predictor.predict(box=box)

    # If you want to visualize or save the masks
    for mask in masks:
        cv2.imshow("Mask", mask)
        cv2.waitKey(0)

### Post-Processing and Visualization

You can post-process the masks to refine them or combine them with the original image. For example, overlaying the masks on the original image or bounding boxes to see the segmentation results.

In [ ]:
# Overlay mask on the original image
overlay_image = image.copy()
for mask in masks:
    overlay_image[mask] = [0, 255, 0]  # Color the masked area

cv2.imshow("Overlay", overlay_image)
cv2.waitKey(0)

### Saving the Results

You can save the segmented images or masks for further use:

In [ ]:
cv2.imwrite("segmented_image.png", overlay_image)